## Week 3 - Neighborhoods in Toronto - Segmenting and Clustering.

Below Code will import the required Libraries for the Project

In [14]:
import numpy as np
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import pandas as pd
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
!conda install -c conda-forge geopy --yes
!pip install geocoder
!pip install bs4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total: 

In [15]:
# import Libraries that we need
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

In [52]:
Web_Page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
Web_Page_Content = BeautifulSoup(Web_Page.content, 'html.parser')
Web_Page_Table = Web_Page_Content.find('tbody')
Web_Page_Table_Rows = Web_Page_Table.select('tr')
Web_Page_Table_Row = [r.get_text() for r in Web_Page_Table_Rows]

In [69]:
Data_Frame = pd.DataFrame(Web_Page_Table_Row)
Data_Frame1 = Data_Frame[0].str.split('\n', expand=True)
Data_Frame2 = Data_Frame1.rename(columns=Data_Frame1.iloc[0])
Data_Frame3 = Data_Frame2.drop(Data_Frame2.index[0])
Data_Frame4 = Data_Frame3[Data_Frame3.Borough != 'Not assigned']
Data_Frame5 = Data_Frame4.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
Data_Frame5.reset_index(inplace = True)
Data_Frame5.Neighbourhood[Data_Frame5.Neighbourhood=="Not assigned"] = Data_Frame5.Borough[Data_Frame5.Neighbourhood=="Not assigned"]
print( Data_Frame5.shape )
Data_Frame5.head()

(103, 3)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [70]:
Data_Frame7 = pd.merge(Data_Frame5, Data_Frame6, on='Postcode')
Data_Frame7.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


## Question 2

In [71]:
url = "http://cocl.us/Geospatial_data"
Data_Frame6 = pd.read_csv(url)
Data_Frame6.rename(columns={'Postal Code': 'Postcode'}, inplace=True) # chane the first column's name to be as the first dataframe
Data_Frame6.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [61]:
print('The dataframe has {} Borough and {} Neighbourhood.'.format(
        len(Data_Frame7['Borough'].unique()),
        Data_Frame7.shape[0]
    )
)

The dataframe has 11 Borough and 103 Neighbourhood.


In [66]:
Data_Frame_Toronto = Data_Frame7[Data_Frame7['Borough'].str.contains('Toronto')]
Data_Frame_Toronto.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [64]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(Data_Frame_Toronto['Latitude'], Data_Frame_Toronto['Longitude'], 
                                           Data_Frame_Toronto['Borough'], Data_Frame_Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map

## Question 3